# Microsoft Agent Framework SDK 테스트

이 노트북은 **Microsoft Agent Framework** (Python SDK)를 사용하여 Azure OpenAI 모델(`gpt-4o-mini`)과 상호 작용하는 방법을 보여줍니다.

## 사전 요구 사항
1. Python 가상 환경(.venv)이 구성되어 있는지 확인하세요.
2. `src/.env` 파일에 Azure OpenAI 연결 정보가 올바르게 설정되어 있는지 확인하세요.


In [20]:
%pip install agent-framework --pre azure-identity python-dotenv azure-ai-projects azure-monitor-opentelemetry azure-search-documents azure-storage-blob openai ipykernel 

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import asyncio
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from agent_framework.azure import AzureOpenAIResponsesClient

# .env 파일 로드
load_dotenv()

# 환경 변수에서 설정 가져오기
azure_ai_service_name = os.getenv("AZURE_AI_SERVICE_NAME")
if azure_ai_service_name and not os.getenv("AZURE_OPENAI_ENDPOINT"):
    azure_openai_endpoint = f"https://{azure_ai_service_name}.openai.azure.com/"
else:
    azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# 배포 이름 (기본값 설정 또는 환경 변수)
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini") 

print(f"Azure OpenAI Endpoint: {azure_openai_endpoint}")
print(f"Deployment Name: {deployment_name}")

Azure OpenAI Endpoint: https://cog-q2kearsbvtc5a.openai.azure.com/
Deployment Name: gpt-4o-mini


In [ ]:
# [Advanced] Telemetry & Project Client Setup
# azure-ai-projects SDK를 사용하여 AI Project에 연결하고 Telemetry(App Insights)를 활성화합니다.
import os

# 로컬 실행 시 발생하는 리소스 감지 오류(StopIteration) 방지를 위해 환경 변수를 최상단에서 설정
os.environ["OTEL_PYTHON_DISABLED_RESOURCE_DETECTORS"] = "azure_app_service,azure_vm"

from azure.ai.projects import AIProjectClient
from azure.ai.projects.telemetry import AIProjectInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.identity import DefaultAzureCredential
from opentelemetry import trace

# AI Project 연결 문자열 (Foundry 포털의 프로젝트 설정에서 확인 가능)
project_connection_string = os.getenv("AZURE_AI_PROJECT_CONNECTION_STRING")

if project_connection_string:
    try:
        # AIProjectClient 초기화
        project_client = AIProjectClient(
            endpoint=project_connection_string,
            credential=DefaultAzureCredential(),
        )
        
        # Application Insights 연결 문자열 가져오기
        app_insights_conn_str = project_client.telemetry.get_application_insights_connection_string()
        if app_insights_conn_str:
            # OpenTelemetry가 이미 구성되어 있는지 확인
            # (기본값인 ProxyTracerProvider 상태인지 확인하여 중복 구성 방지)
            current_provider = trace.get_tracer_provider()
            
            # ProxyTracerProvider인 경우에만 구성을 진행
            if "ProxyTracerProvider" in type(current_provider).__name__:
                print("Configuring Azure Monitor OpenTelemetry...")
                configure_azure_monitor(connection_string=app_insights_conn_str)
                AIProjectInstrumentor().instrument()
                print("Application Insights Tracing enabled.")
            else:
                print("OpenTelemetry is already configured. Skipping re-initialization to avoid errors.")
        else:
            print("Application Insights not configured for this project.")
            
    except Exception as e:
        print(f"Failed to initialize AIProjectClient or Telemetry: {e}")
else:
    print("AZURE_AI_PROJECT_CONNECTION_STRING not found in .env. Telemetry setup skipped.")

[2026-01-20 09:22:43,153] Failed to load resource detector 'azure_app_service', skipping
Traceback (most recent call last):
  File "c:\Users\tjpark\AppData\Local\Programs\Python\Python313\Lib\site-packages\opentelemetry\sdk\resources\__init__.py", line 214, in create
    next(
    ~~~~^
        iter(
        ^^^^^
    ...<4 lines>...
        )
        ^
    ).load()()
    ^
StopIteration
[2026-01-20 09:22:43,155] Failed to load resource detector 'azure_vm', skipping
Traceback (most recent call last):
  File "c:\Users\tjpark\AppData\Local\Programs\Python\Python313\Lib\site-packages\opentelemetry\sdk\resources\__init__.py", line 214, in create
    next(
    ~~~~^
        iter(
        ^^^^^
    ...<4 lines>...
        )
        ^
    ).load()()
    ^
StopIteration
[2026-01-20 09:22:43,203] Overriding of current TracerProvider is not allowed
[2026-01-20 09:22:43,236] Overriding of current LoggerProvider is not allowed
c:\Users\tjpark\AppData\Local\Programs\Python\Python313\Lib\site-packag

Application Insights Tracing enabled.


In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import os

conn_str = os.getenv("AZURE_AI_PROJECT_CONNECTION_STRING")
try:
    client = AIProjectClient(endpoint=conn_str, credential=DefaultAzureCredential())
    print("Instance created.")
    print(dir(client))
    if hasattr(client, 'telemetry'):
        print("Telemetry attribute found.")
    else:
        print("Telemetry attribute NOT found.")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# 기본 에이전트 예제: 수필 ChatBot
# 'AzureOpenAIResponsesClient'를 사용하여 상태를 저장하지 않는(Stateless) 가벼운 에이전트를 생성합니다.
# 이 방식은 복잡한 문맥 관리가 필요 없는 단순한 질의응답이나 일회성 작업에 적합합니다.

async def run_haiku_bot():
    try:
        # Azure OpenAI 클라이언트 생성
        # Managed Agent와 달리 OpenAI 모델 엔드포인트에 직접 연결하여 추론을 수행합니다.
        client = AzureOpenAIResponsesClient(
            deployment_name=deployment_name,
            endpoint=azure_openai_endpoint,
            credential=DefaultAzureCredential(),
        )
        
        # 에이전트 정의 (create_agent 메서드 사용)
        # Managed Agent처럼 서버에 등록되지 않고, 로컬 메모리상에서만 존재하는 객체입니다.
        agent = client.create_agent(
            name="essayBot",
            instructions="당신은 아름다운 글을 쓰는 긍정적인 도우미입니다.",
        )

        print(f"에이전트 '{agent.name}'가 생성되었습니다.")

        # 에이전트 실행 및 응답 대기
        input_text = "연못 위의 오리로 시작하는 짧은 수필을 써주세요."
        print(f"\n입력: {input_text}\n")
        
        response = await agent.run(input_text)
        print(f"응답:\n{response}")

    except Exception as e:
        print(f"에이전트 실행 중 오류 발생: {e}")
        # print("환경 변수(Endpoint 등)가 올바르게 설정되었는지 확인하세요.") # Exception message is usually enough

await run_haiku_bot()

In [ ]:
# [Advanced] Azure AI Search Integration
# example/search_index_manager.py의 로직을 참조하여 Azure AI Search 서비스에 연결합니다.

from azure.search.documents.indexes import SearchIndexClient

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") # .env에 추가 필요
# 또는 AI Project를 통해 검색 리소스 정보를 가져올 수도 있습니다 (SDK 지원 시)

if search_endpoint:
    try:
        search_credential = DefaultAzureCredential()
        index_client = SearchIndexClient(endpoint=search_endpoint, credential=search_credential)
        
        # 인덱스 목록 조회
        print("Existing Search Indexes:")
        names = [x.name for x in index_client.list_indexes()]
        for name in names:
            print(f"- {name}")
            
    except Exception as e:
        print(f"Error accessing Azure AI Search: {e}")
else:
    print("AZURE_SEARCH_ENDPOINT not set. Search integration skipped.")

In [ ]:
# [Basic] Project 정보 및 Connections 확인
# AI Project에 연결된 리소스(Connection)들을 확인합니다.
# 이를 통해 현재 프로젝트가 어떤 리소스(OpenAI, Search 등)와 연결되어 있는지 파악할 수 있습니다.

if 'project_client' in locals() and project_client:
    # Scope 속성이 없으므로 제거하고 대신 기본적인 정보 출력 (없으면 생략)
    # print(f"Project Scope: {project_client.scope}\n")
    
    print("--- Connections List ---")
    connections = project_client.connections.list()
    for conn in connections:
        # 연결의 이름과 유형을 출력 (예: AzureOpenAI, AzureSearch 등)
        print(f"Name: {conn.name:<30} Type: {conn.type}")
else:
    print("Project Client가 초기화되지 않았습니다. 상단의 'Telemetry & Project Client Setup' 셀을 실행하세요.")

In [ ]:
if 'project_client' in locals() and project_client:
    print("Project Client Dir:", dir(project_client))
    print("Agents Operations Dir:", dir(project_client.agents))

In [ ]:
# [Intermediate] Managed Agent 생성 및 실행 (Agents API)
# Azure AI Agent Service (Managed)를 사용하여 에이전트를 생성하고 대화합니다.
# 이 방식은 서버 측에서 상태(State)를 관리하는 Assistants API를 사용합니다.

from azure.ai.agents import AgentsClient
import time

if 'project_connection_string' in locals() and project_connection_string:
    try:
        # AgentsClient 초기화
        # project_connection_string은 Azure AI Foundry 프로젝트의 연결 문자열입니다.
        # 이 클라이언트는 에이전트 생성, 스레드 관리, 실행(Run) 등을 담당합니다.
        agents_client = AgentsClient(project_connection_string, credential=DefaultAzureCredential())
        
        print("Creating Agent...")
        # 1. 에이전트 생성
        # create_agent는 서버에 에이전트 구성을 저장합니다.
        # instructions: 시스템 프롬프트 역할을 하며 에이전트의 성격과 행동을 정의합니다.
        agent = agents_client.create_agent(
            model=deployment_name,
            name="my-managed-agent",
            instructions="당신은 도움이 되는 AI 비서입니다. 간결하게 답변해주세요.",
        )
        print(f"Created agent, ID: {agent.id}")

        # 2. 스레드 생성 (대화 문맥 관리)
        # Thread는 사용자와 에이전트 간의 대화 기록(Message History)을 저장하는 컨테이너입니다.
        # 서버에서 관리되므로 클라이언트가 전체 대화 기록을 유지할 필요가 없습니다.
        thread = agents_client.threads.create()
        print(f"Created thread, ID: {thread.id}")

        # 3. 메시지 추가
        # 생성된 스레드에 사용자의 질문을 추가합니다.
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="안녕하세요! 우주에 대한 재미있는 사실 하나만 알려주세요.", # 한글 질문
        )
        print(f"Created message, ID: {message.id}")

        # 4. 실행 (Run)
        # Run은 특정 스레드의 대화 내용을 바탕으로 에이전트를 실행하는 작업입니다.
        # 비동기로 동작하므로 실행 상태를 주기적으로 확인해야 합니다.
        print("Starting Run...")
        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

        # 5. 상태 확인 (Polling)
        # 실행이 완료(completed), 실패(failed), 또는 취소(cancelled)될 때까지 상태를 확인합니다.
        previous_status = None
        while run.status in ["queued", "in_progress", "requires_action"]:
            if run.status != previous_status:
                print(f"Run Status: {run.status}")
                previous_status = run.status
            time.sleep(1) # 1초 대기 후 상태 재확인
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        print(f"Run Completed with status: {run.status}")

        if run.status == "failed":
            print(f"Run Failed Key: {run.last_error}")

        # 6. 응답 확인
        # 실행이 성공하면 스레드에 에이전트의 응답 메시지가 추가됩니다.
        # 메시지 목록을 가져와서 출력합니다.
        if run.status == "completed":
            messages = agents_client.messages.list(thread_id=thread.id)
            print("\n--- Conversation ---")
            msg_list = list(messages)
            # 메시지는 최신순으로 반환되므로 역순으로 출력하여 대화 흐름대로 보여줍니다.
            for msg in reversed(msg_list):
                for content in msg.content:
                    if hasattr(content, 'text'):
                        print(f"[{msg.role.upper()}]: {content.text.value}")

        # 7. 정리 (옵션)
        # 생성된 에이전트를 삭제하여 리소스를 정리할 수 있습니다.
        # agents_client.delete_agent(agent.id)

    except Exception as e:
        print(f"Error using managed agent: {e}")
else:
    print("Project Connection String이 설정되지 않았습니다.")

In [ ]:
# [Advanced] Managed Agent with Code Interpreter
# 코드 인터프리터 도구를 사용하여 수학 계산, 데이터 분석 등을 수행하는 에이전트 예제입니다.
# Managed Agent는 서버 측 샌드박스 환경에서 파이썬 코드를 실행할 수 있습니다.

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import CodeInterpreterToolDefinition

if 'project_connection_string' in locals() and project_connection_string:
    try:
        agents_client = AgentsClient(project_connection_string, credential=DefaultAzureCredential())
        print("Creating Math Agent with Code Interpreter...")
        
        # 1. 코드 인터프리터 도구 정의
        # 이 도구를 사용하면 에이전트가 Python 코드를 작성하고 실행하여 결과를 얻을 수 있습니다.
        code_interpreter = CodeInterpreterToolDefinition()

        # 2. 에이전트 생성 (tools 파라미터에 정의 전달)
        # tools 리스트에 code_interpreter를 추가하여 에이전트에게 해당 기능을 부여합니다.
        math_agent = agents_client.create_agent(
            model=deployment_name,
            name="math-agent",
            # 한글 지시사항으로 변경: 수학자로서 코드를 사용하여 문제를 해결하도록 지시
            instructions="당신은 수학자입니다. 코드 인터프리터를 사용하여 수학 문제를 해결하세요. 항상 코드를 설명해주세요.",
            tools=[code_interpreter] 
        )
        print(f"Created math agent, ID: {math_agent.id}")

        # 3. 스레드 및 메시지
        thread = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            # 피보나치 수열 계산 요청 (한글)
            content="피보나치 수열의 첫 5개 숫자를 코드를 작성해서 계산해주세요.",
        )

        # 4. 실행 및 폴링
        print("Starting Run...")
        run = agents_client.runs.create(thread_id=thread.id, agent_id=math_agent.id)
        
        previous_status = None
        while run.status in ["queued", "in_progress", "requires_action"]:
            if run.status != previous_status:
                print(f"Run Status: {run.status}")
                previous_status = run.status
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        print(f"Run Completed with status: {run.status}")

        if run.status == "failed":
            print(f"Run Failed Error: {run.last_error}")

        # 5. 결과 확인 (코드 및 출력 포함)
        if run.status == "completed":
            messages = agents_client.messages.list(thread_id=thread.id)
            print("\n--- Conversation with Code ---")
            msg_list = list(messages)
            for msg in reversed(msg_list):
                print(f"\n[{msg.role.upper()}]")
                for content in msg.content:
                    if hasattr(content, 'text'):
                        print(content.text.value)
                    # 에이전트가 코드를 실행하여 이미지를 생성한 경우 (예: 그래프) image_file 속성이 존재합니다.
                    elif hasattr(content, 'image_file'):
                        print("[Image File Created]")
                    
        # 6. 정리
        # agents_client.delete_agent(math_agent.id)

    except Exception as e:
        print(f"Error with Code Interpreter: {e}")
else:
    print("Project Connection String이 설정되지 않았습니다.")

In [ ]:
# [Advanced] Managed Agent with Azure AI Search (RAG)
# Azure AI Search를 에이전트의 지식 베이스(Knowledge Base)로 활용하는 예제입니다.

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import AzureAISearchToolDefinition

if 'project_client' in locals() and project_client:
    try:
        # 1. Azure AI Search 연결 확인 (Project Client 사용)
        connections = project_client.connections.list()
        search_conn = None
        for conn in connections:
            # Type check: Might be ConnectionType.AZURE_SEARCH or similar
            if 'search' in conn.type.lower() or 'search' in conn.name.lower():
                search_conn = conn
                break
        
        if search_conn:
            print(f"Found Search Connection: {search_conn.name} (ID: {search_conn.id})")
            
            # Agents Client 초기화
            agents_client = AgentsClient(project_connection_string, credential=DefaultAzureCredential())

            # 인덱스 이름 (사전에 생성되어 있어야 함)
            index_name = "my-search-index"
            
            # 2. Search 도구 정의
            search_tool = AzureAISearchToolDefinition(
                index_connection_id=search_conn.id,
                index_name=index_name
            )
            
            # 3. 에이전트 생성 (검색 도구 포함)
            rag_agent = agents_client.create_agent(
                model=deployment_name,
                name="search-agent",
                instructions="You are a helpful assistant. Use the search tool to answer questions based on the provided data.",
                tools=[search_tool]
            )
            print(f"Created Search Agent, ID: {rag_agent.id}")
            
            # 4. 실행 (검색이 필요한 질문)
            thread = agents_client.threads.create()
            agents_client.messages.create(
                thread_id=thread.id,
                role="user",
                # 실제 인덱스가 없으면 검색이 실패할 수 있으니 일반적인 질문으로 변경하거나 예외 처리 필요
                content="What is the refund policy according to the search data?",
            )
            
            print("Starting Run...")
            run = agents_client.runs.create(thread_id=thread.id, agent_id=rag_agent.id)
            
            # ... (Polling 및 결과 확인 로직은 동일) ...
            previous_status = None
            while run.status in ["queued", "in_progress", "requires_action"]:
                if run.status != previous_status:
                    print(f"Run Status: {run.status}")
                    previous_status = run.status
                time.sleep(1)
                run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

            print(f"Run Completed with status: {run.status}")
            if run.status == "failed":
                print(f"Run Failed Key: {run.last_error}")
            
            # 메시지 출력
            # ...
            
        else:
            print("Azure AI Search 연결을 찾을 수 없습니다.")

    except Exception as e:
        print(f"Error with Azure AI Search Agent: {e}")
else:
    print("Project Client가 초기화되지 않았습니다.")

In [ ]:
import agent_framework
import pkgutil

print("Scanning for multi-agent related modules in agent_framework...")
found = []
for loader, module_name, is_pkg in pkgutil.walk_packages(agent_framework.__path__):
    if any(keyword in module_name for keyword in ['multi', 'group', 'orchestrat', 'route', 'team', 'chat']):
        found.append(module_name)
        print(f" - {module_name}")

if not found:
    print("No obvious multi-agent modules found. Constructing manual loop example.")

In [ ]:
# [Advanced] Multi-Agent Orchestration (HR Onboarding Scenario)
# HR 담당자(HR Specialist)와 부서 리드(Department Lead) 에이전트가 협업하여 신규 입사자 온보딩 계획을 수립하는 시나리오입니다.
# Managed Agent Service는 기본적으로 단일 에이전트 실행 모델입니다. 
# 여러 에이전트가 협업하기 위해서는 클라이언트 코드(Python)에서 대화의 흐름(Workflow)을 제어해야 합니다.

from azure.ai.agents import AgentsClient
import time

if 'project_connection_string' in locals() and project_connection_string:
    try:
        agents_client = AgentsClient(project_connection_string, credential=DefaultAzureCredential())

        # 1. 에이전트 생성
        print("HR 워크플로우를 위한 에이전트 생성 중...")
        
        # HR 담당자 에이전트 정의
        # 역할: 온보딩 계획의 전반적인 구조와 회사 정책 관련 내용을 담당
        hr_agent = agents_client.create_agent(
            model=deployment_name,
            name="HR_Specialist",
            instructions="당신은 HR 담당자입니다. 신규 입사자를 위한 포괄적이고 환영받는 온보딩 계획을 수립하는 것이 당신의 책임입니다. 회사 정책과 일반적인 회사 소개가 포함되도록 하세요.",
        )
        
        # 부서 리드 에이전트 정의
        # 역할: 기술적인 준비 사항과 팀 적응 관련 내용을 검토 및 피드백
        lead_agent = agents_client.create_agent(
            model=deployment_name,
            name="Dept_Lead",
            instructions="당신은 엔지니어링 부서 리드입니다. 온보딩 계획에 필요한 기술적 설정(노트북, 접근 권한), 팀 소개, 관련 기술 교육이 포함되어 있는지 검토하세요.",
        )
        
        print(f"생성된 에이전트: {hr_agent.name}, {lead_agent.name}")

        # 2. 협업 루프 (Workflow)
        # 주제: "Senior Software Engineer 온보딩"
        new_hire_role = "Senior Software Engineer"
        start_date = "다음 주 월요일"
        print(f"\n--- 온보딩 워크플로우 시작: '{new_hire_role}' (시작일: {start_date}) ---")

        # Helper 함수: 에이전트 실행 및 응답 대기
        # 반복되는 "Run 생성 -> Polling -> 결과 조회" 패턴을 함수화했습니다.
        def run_agent(agent_obj, thread_obj):
            run = agents_client.runs.create(thread_id=thread_obj.id, agent_id=agent_obj.id)
            
            # Polling loop (실행 완료 대기)
            while run.status in ["queued", "in_progress", "requires_action"]:
                time.sleep(1)
                run = agents_client.runs.get(thread_id=thread_obj.id, run_id=run.id)
            
            if run.status == "completed":
                messages = agents_client.messages.list(thread_id=thread_obj.id)
                msg_list = list(messages)
                # 최신 메시지 반환 (보통 리스트의 첫 번째 요소가 최신 메시지임)
                for msg in msg_list: 
                     for content in msg.content:
                        if hasattr(content, 'text'):
                            return content.text.value
            else:
                raise Exception(f"실행 실패 상태: {run.status}, 오류: {run.last_error}")

        # [Step 1] HR 담당자가 초안 작성
        # HR 에이전트를 위한 전용 스레드를 생성하여 작업을 지시합니다.
        print("\n[Step 1] HR 담당자가 온보딩 계획 초안을 작성 중입니다...")
        thread_hr = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread_hr.id,
            role="user",
            content=f"{start_date}에 시작하는 신규 {new_hire_role}를 위한 3일간의 온보딩 일정을 작성해 주세요. 회사 문화와 일반적인 설정에 초점을 맞추세요.",
        )
        
        draft_plan = run_agent(hr_agent, thread_hr)
        print(f"초안 계획:\n{draft_plan}\n")

        # [Step 2] 부서 리드가 기술적 관점에서 검토 및 피드백 제공
        # 부서 리드를 위한 새로운 스레드를 생성하고, HR의 초안을 입력으로 전달합니다.
        print("[Step 2] 부서 리드가 기술적 요구사항을 검토 중입니다...")
        thread_lead = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread_lead.id,
            role="user",
            content=f"다음 {new_hire_role}를 위한 온보딩 계획을 검토해 주세요. 누락된 기술적 설정(GitHub 접근 권한, 개발 환경)과 팀별 활동을 지적해 주세요:\n\n{draft_plan}",
        )
        
        feedback = run_agent(lead_agent, thread_lead)
        print(f"리드 피드백:\n{feedback}\n")

        # [Step 3] HR 담당자가 피드백을 반영하여 최종 계획 수립
        # HR 에이전트의 기존 스레드에 피드백을 추가하여 문맥을 유지한 채로 수정을 요청합니다.
        print("[Step 3] HR 담당자가 계획을 최종 확정 중입니다...")
        agents_client.messages.create(
            thread_id=thread_hr.id,
            role="user",
            content=f"부서 리드의 다음 피드백을 반영하여 일정을 업데이트해 주세요:\n\n{feedback}",
        )
        
        final_plan = run_agent(hr_agent, thread_hr)
        print(f"최종 온보딩 계획:\n{final_plan}\n")
        
        # 3. 정리 (옵션)
        # agents_client.delete_agent(hr_agent.id)
        # agents_client.delete_agent(lead_agent.id)

    except Exception as e:
        print(f"HR 온보딩 워크플로우 오류: {e}")
else:
    print("Project Connection String이 설정되지 않았습니다.")

In [ ]:
import azure.ai.agents.models
try:
    from azure.ai.agents.models import CodeInterpreterToolDefinition
    print("CodeInterpreterToolDefinition found")
except ImportError:
    print("CodeInterpreterToolDefinition NOT found")

try:
    from azure.ai.agents.models import AzureAISearchToolDefinition
    print("AzureAISearchToolDefinition found")
except ImportError:
    print("AzureAISearchToolDefinition NOT found")